# 贪心策略

贪心策略一般在复杂度和实现上都比较优越，而且需要证明正确性（在比赛中一般靠直觉）。解题时，可以先思考一种简单但是不够高效的算法，在再凭借直觉改进为贪心算法。
有些题目主要运用了数学推导以降低时间复杂度，但是算作贪心亦可。

- [1013 将数组分成相等的三个部分](https://leetcode-cn.com/problems/partition-array-into-three-parts-with-equal-sum/)
- [1071 字符串的最大公因子](https://leetcode-cn.com/problems/greatest-common-divisor-of-strings/)
- [300 最长上升子序列](https://leetcode-cn.com/problems/longest-increasing-subsequence/)

## 将数组分成相等的三个部分

这道题可以很快想到双指针的方法，时间复杂度为$O(n^2)$，

In [1]:
def canThreePartsEqualSum(A) -> bool:
    def step_add(A):
        n = len(A)
        b = [0 for _ in range(n)]
        b[0] = A[0]
        for i in range(1, n):
            b[i] = b[i - 1] + A[i]

        return b

    n = len(A)
    all_sum = sum(A)
    left_sum = step_add(A)
    right_sum = step_add(A[::-1])

    for i in range(n - 2):
        for j in range(n - 2 - i):
            if left_sum[i] == right_sum[j] and left_sum[i] == (all_sum - left_sum[i] - right_sum[j]):
                return True
    return False


print(canThreePartsEqualSum([0, 2, 1, -6, 6, -7, 9, 1, 2, 0, 1]))

True


这样是会超时，可以通过以下两点剪枝：
1. 都是整数，如果数组的和不是3的倍数，则不能三等分
2. 每个部分都必须等于 sum / 3

In [2]:
def canThreePartsEqualSum(A) -> bool:
    def step_add(A):
        n = len(A)
        b = [0 for _ in range(n)]
        b[0] = A[0]
        for i in range(1, n):
            b[i] = b[i - 1] + A[i]
        return b

    n = len(A)
    all_sum = sum(A)

    if all_sum % 3 != 0:
        return False

    one_three_sum = all_sum // 3
    left_sum = step_add(A)
    right_sum = step_add(A[::-1])
    for i in range(n - 2):
        if left_sum[i] == one_three_sum:
            for j in range(n - 2 - i):
                if right_sum[j] == one_three_sum:
                    return True
    return False

print(canThreePartsEqualSum([0, 2, 1, -6, 6, -7, 9, 1, 2, 0, 1]))

True


到这里，已经AC。但是，这道题可以通过贪心策略实现O(n)的时间复杂度，并且迁移到任意等分。

以k等分为例，从左往右累加，当和为$\frac{sum}{k}$，累加和清零后继续，遍历中，如果存在k个等分点（或存在k-1个等分点且尚未遍历到数组末尾），即表示可k等分。

In [3]:
def canThreePartsEqualSum(A) -> bool:
        n = len(A)
        all_sum = sum(A)

        if all_sum % 3 != 0:
            return False

        one_three_sum = all_sum // 3
        k = 0
        part_sum = 0
        for a in A:
            part_sum += a
            if part_sum == one_three_sum:
                k += 1
                part_sum = 0
            if k == 3:
                return True
        return False
    
print(canThreePartsEqualSum([0, 2, 1, -6, 6, -7, 9, 1, 2, 0, 1]))

True


## 字符串的最大公因子
这道题目不难，枚举也可以AC，但是题解中的方法非常秀。

[题解-证明](https://leetcode-cn.com/problems/greatest-common-divisor-of-strings/solution/zi-fu-chuan-de-zui-da-gong-yin-zi-by-leetcode-solu/)


In [4]:
import math

def gcdOfStrings(str1: str, str2: str) -> str:
    gcd = math.gcd(len(str1), len(str2))
    if str1[0:gcd]*(len(str1)//gcd) == str1 and str1[0:gcd]*(len(str2)//gcd) == str2:
        return str1[0:gcd]
    return ""

print(gcdOfStrings(str1="ABCABC", str2="ABC"))

ABC


这还不够，还可以更秀，复杂度$\downnarrow$，代码行数$\downnarrow$。

贪心策略：

In [8]:
import math

def gcdOfStrings(str1: str, str2: str) -> str:
    gcd = math.gcd(len(str1), len(str2))
    if str1 + str2 == str2 + str1:
        return str1[0:gcd]
    return ""
print(gcdOfStrings(str1="ABCABC", str2="ABC"))

def gcdOfStringsOneLineCode(str1: str, str2: str) -> str:
    return str1[0: math.gcd(len(str1), len(str2))] if str1 + str2 == str2 + str1 else ""
print(gcdOfStringsOneLineCode(str1="ABCABC", str2="ABC"))


ABC
ABC


## 最长上升子序列
可以简单的设计一个$O(n^2)$的动态规划算法，但是“稍加”思索，我们就可以得到$O(nlogn)$的算法。

贪心策略：要使上升子序列尽可能的长，则需要让序列上升得尽可能慢，因此希望每次在上升子序列最后加上的那个数尽可能的小。

数组tmp[i]存储长度为$i+1$的序列的最后一个元素，遍历数字$nums$，如果：
1. if nums[i] > tmp[-1]，说明出现了长度为 len(tmp) + 1 的序列，tmp.append(nums[i])
2. else，某个长度的序列最后一个元素需要被修改，可用二分查找

In [9]:
def lengthOfLIS(nums) -> int:
    import bisect
    tmp = []
    for i in range(len(nums)):
        j = bisect.bisect_left(tmp, nums[i])
        if j >= len(tmp):
            tmp.append(nums[i])
        else:
            tmp[j] = nums[i]
    return len(tmp)

lengthOfLIS(nums=[10,9,2,5,3,7,101,18])       


4